In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
from util.util import print_progress
from util.create_dataset import create_dataset, get_batch
from util.midi_manipulation import noteStateMatrixToMidi

In [3]:
min_song_length  = 128
encoded_songs    = create_dataset(min_song_length)

88 songs processed
15 songs discarded


In [4]:
NUM_SONGS = len(encoded_songs)
print(str(NUM_SONGS) + " total songs to learn from")
print(encoded_songs[0].shape)

73 total songs to learn from
(257, 78)


In [5]:
input_size       = encoded_songs[0].shape[1]
output_size      = input_size         
hidden_size      = 128 

In [6]:
learning_rate    = 0.001
training_steps   = 200
batch_size       = 256
timesteps        = 64

In [7]:
assert timesteps < min_song_length

In [8]:
input_placeholder_shape = [None, timesteps, input_size]
output_placeholder_shape = [None, output_size]

In [9]:
input_vec  = tf.placeholder("float", input_placeholder_shape)  
output_vec = tf.placeholder("float", output_placeholder_shape)

In [10]:
weights = tf.Variable(tf.random_normal([hidden_size, output_size]))

In [11]:
biases = tf.Variable(tf.random_normal([output_size]))

In [12]:
def RNN(input_vec, weights, biases):
    input_vec = tf.unstack(input_vec, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(hidden_size) # TODO 
    outputs, states = rnn.static_rnn(lstm_cell, input_vec, dtype=tf.float32)
    recurrent_net = tf.matmul(outputs[-1], weights) + biases # TODO 
    prediction = tf.nn.softmax(recurrent_net) # TODO
    return recurrent_net, prediction

In [13]:
logits, prediction = RNN(input_vec, weights, biases)

In [14]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=output_vec))

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) # TODO
train_op = optimizer.minimize(loss_op)

In [16]:
true_note = tf.argmax(output_vec,1)
pred_note = tf.argmax(prediction, 1)
correct_pred = tf.equal(pred_note, true_note)

In [17]:
accuracy_op = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [18]:
init = tf.global_variables_initializer()

In [19]:
sess = tf.InteractiveSession()

In [20]:
sess.run(init)

In [21]:
display_step = 1

In [22]:
for step in range(training_steps):
    batch_x, batch_y = get_batch(encoded_songs, batch_size, timesteps, input_size, output_size) # TODO

    feed_dict = {
                    input_vec: batch_x, # TODO remove after colon
                    output_vec: batch_y # TODO remove after colon
                }
    sess.run(train_op, feed_dict=feed_dict)
    
    if step % display_step == 0 or step == 1:
        
        loss, acc = sess.run([loss_op, accuracy_op], feed_dict=feed_dict)     
        suffix = "\nStep " + str(step) + ", Minibatch Loss= " + \
                 "{:.4f}".format(loss) + ", Training Accuracy= " + \
                 "{:.3f}".format(acc)

        print_progress(step, training_steps, barLength=50, suffix=suffix)

 |--------------------------------------------------| 0.0% 
 |--------------------------------------------------| 0.5% 
 |=-------------------------------------------------| 1.0% 
 |=-------------------------------------------------| 1.5% 
 |=-------------------------------------------------| 2.0% 
 |=-------------------------------------------------| 2.5% 
 |==------------------------------------------------| 3.0% 
 |==------------------------------------------------| 3.5% 
 |==------------------------------------------------| 4.0% 
 |==------------------------------------------------| 4.5% 
 |===-----------------------------------------------| 5.0% 
 |===-----------------------------------------------| 5.5% 
 |===-----------------------------------------------| 6.0% 
 |===-----------------------------------------------| 6.5% 
 |====----------------------------------------------| 7.0% 
 |====----------------------------------------------| 7.5% 
 |====----------------------------------

 |==================================----------------| 67.0% 
 |==================================----------------| 67.5% 
 |==================================----------------| 68.0% 
 |==================================----------------| 68.5% 
 |===================================---------------| 69.0% 
 |===================================---------------| 69.5% 
 |===================================---------------| 70.0% 
 |===================================---------------| 70.5% 
 |====================================--------------| 71.0% 
 |====================================--------------| 71.5% 
 |====================================--------------| 72.0% 
 |====================================--------------| 72.5% 
 |=====================================-------------| 73.0% 
 |=====================================-------------| 73.5% 
 |=====================================-------------| 74.0% 
 |=====================================-------------| 74.5% 
 |======================

In [23]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
GEN_SEED_RANDOMLY = False

In [24]:
if GEN_SEED_RANDOMLY:
    ind = np.random.randint(NUM_SONGS)
else:
    ind = 41

In [25]:
gen_song = encoded_songs[ind][:timesteps].tolist()

In [27]:
for i in range(100):
    seed = np.array([gen_song[-timesteps:]])
    predict_probs = sess.run(prediction, feed_dict = {input_vec:seed})
    played_notes = np.zeros(output_size) 
    #print(np.argmax(predict_probs[0]))
    plt.plot(predict_probs[0])
    sampled_note = np.random.choice(range(output_size), p=predict_probs[0])
    played_notes[sampled_note] = 1
    gen_song.append(played_notes)

In [28]:
noteStateMatrixToMidi(gen_song, name="generated/gen_song_0")
noteStateMatrixToMidi(encoded_songs[ind], name="generated/base_song_0")
print("saved generated song!".format(ind))

saved generated song!
